### Inlämningsuppgift Prediktiv analys
### Tatiana Ilyasova, 2022-03-18

In [1]:
import pandas as pd
import numpy as np

from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2

import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', None) 
pio.templates.default = 'plotly_white'

Funktioner som underlättar vidare att 1. Beräkna R2, RMSE och MAE; 2. Beräkna R2, RMSE och MAE om 'target' är Log-transformed;
3. Lägga till och uppdatera data i tabellen med alla metrics; 4. Printa ut resultat 

In [2]:
def mod_metrics(model):
    score_train, score_test = model.score(X_train, y_train).round(4), model.score(X_test, y_test).round(4)
    RMSE_train = MSE(y_train, model.predict(X_train),squared=False).round(0)
    RMSE_test = MSE(y_test, model.predict(X_test),squared=False).round(0)
    MAE_train = MAE(y_train, model.predict(X_train)).round(0)
    MAE_test = MAE(y_test, model.predict(X_test)).round(0)
    
    return(score_train, score_test, RMSE_train, RMSE_test, MAE_train, MAE_test)

def mod_metrics_log(model):
    score_train, score_test = model.score(X_train, y_train_log).round(4), model.score(X_test, y_test_log).round(4)
    RMSE_train = MSE(np.exp(y_train_log), np.exp(model.predict(X_train)),squared=False).round(0)
    RMSE_test = MSE(np.exp(y_test_log), np.exp(model.predict(X_test)),squared=False).round(0)
    MAE_train = MAE(np.exp(y_train_log), np.exp(model.predict(X_train))).round(0)
    MAE_test = MAE(np.exp(y_test_log), np.exp(model.predict(X_test))).round(0)

    return(score_train, score_test, RMSE_train, RMSE_test, MAE_train, MAE_test)

def metrics_df(col, model, log='n'):
    if log == 'n':
        df_metrics[col] = mod_metrics(model)
    else:
        df_metrics[col] = mod_metrics_log(model)
    return(df_metrics)
    
def output_metrics(col):
    print('Training R2: {0:.4f} vs Test R2: {1:.4f}'.format(
        df_metrics.loc['R2_train',col], df_metrics.loc['R2_test',col]))
    print('Training RMSE: {0:.0f} vs Test RMSE: {1:.0f}'.format(
        df_metrics.loc['RMSE_train', col], df_metrics.loc['RMSE_test',col]))
    print('Training MAE: {0:.0f} vs Test MAE: {1:.0f}'.format(
        df_metrics.loc['MAE_train', col], df_metrics.loc['MAE_test',col]))

## Statistics and EDA
### Load and verify the data as-is

In [3]:
df = pd.read_csv('prediktiv_data.csv', index_col = 'id')
df.head(3)

,target,feature01,feature02,feature03,feature04,feature05,feature06,feature07,feature08,feature09,feature10,feature10.1,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23
id,,,,,,,,,,,,,,,,,,,,,,,,,
1,215000,2.0,528.0,0,1080.0,1656,7,1656,6,5,1960,1,NaN,NaN,red,2,good,bad,3,1,5,2010,0,0,1960
2,105000,1.0,730.0,0,882.0,896,5,896,5,6,1961,1,NaN,dog,red,0,NaN,bad,2,1,6,2010,0,0,1961
3,172000,1.0,312.0,0,1329.0,1329,6,1329,6,6,1958,1,NaN,NaN,red,0,NaN,good,3,1,6,2010,1,0,1958


In [4]:
df.shape

(2930, 25)

In [5]:
df.describe()

,target,feature01,feature02,feature03,feature04,feature05,feature06,feature07,feature08,feature09,feature10,feature10.1,feature14,feature17,feature18,feature19,feature20,feature21,feature22,feature23
count,2930.000000,2929.000000,2929.000000,2930.000000,2929.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000
mean,180796.060068,1.766815,472.819734,2.243345,1051.614544,1159.557679,6.443003,1499.690444,6.094881,5.563140,1971.356314,1.566553,0.599317,2.854266,1.044369,6.216041,2007.790444,0.379522,335.455973,1984.266553
std,79886.692357,0.760566,215.046549,35.597181,440.615067,391.890885,1.572964,505.508887,1.411026,1.111537,30.245361,0.552941,0.647921,0.827731,0.214076,2.714492,1.316613,0.502629,428.395715,20.860286
min,12789.000000,0.000000,0.000000,0.000000,0.000000,334.000000,2.000000,334.000000,1.000000,1.000000,1872.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,0.000000,0.000000,1950.000000
25%,129500.000000,1.000000,320.000000,0.000000,793.000000,876.250000,5.000000,1126.000000,5.000000,5.000000,1954.000000,1.000000,0.000000,2.000000,1.000000,4.000000,2007.000000,0.000000,0.000000,1965.000000
50%,160000.000000,2.000000,480.000000,0.000000,990.000000,1084.000000,6.000000,1442.000000,6.000000,5.000000,1973.000000,2.000000,1.000000,3.000000,1.000000,6.000000,2008.000000,0.000000,0.000000,1993.000000
75%,213500.000000,2.000000,576.000000,0.000000,1302.000000,1384.000000,7.000000,1742.750000,7.000000,6.000000,2001.000000,2.000000,1.000000,3.000000,1.000000,8.000000,2009.000000,1.000000,703.750000,2004.000000
max,755000.000000,5.000000,1488.000000,800.000000,6110.000000,5095.000000,15.000000,5642.000000,10.000000,9.000000,2010.000000,4.000000,4.000000,8.000000,3.000000,12.000000,2010.000000,2.000000,2065.000000,2010.000000


In [6]:
df.describe(include='object')

,feature11,feature12,feature13,feature15,feature16
count,198,572,2929,1508,2930
unique,2,4,5,5,5
top,Grvl,dog,red,good,bad
freq,120,330,2682,744,1494


Dataset har 2930 observationer, en target och 24 features, fem av dem är kategoska. Target är kvantativ kontinuerlig variable (det vet vi inte 100% men den ser ut som kontinuerlig). Modeller som kan skapas för att prediktera target är Linear Regression, K-nearest Neighbors, Decision Tree, Random Forest, Gradient Boosting.

In [7]:
target_mean = df['target'].mean()
target_meadian = df['target'].median()

fig = px.histogram(df, x='target', marginal='box', nbins=50,
            color_discrete_sequence=['steelblue'], title = 'Distribution and Boxplot of Target')
fig.add_vline(x=target_mean, 
                line_width=2, line_dash='dash', line_color='green')
fig.add_vline(x=target_meadian, 
                line_width=2, line_dash='dash', line_color='darkgreen')

fig.add_annotation(x=target_mean + 18000, y=500,
            text='Mean',
            showarrow=False
            )
fig.add_annotation(x=target_meadian - 22000, y=500,
            text='Median',
            showarrow=False
            )
fig

Vi ser att mean är högre än median och det finns 'svansen' åt höger. Targets distribution är positiv/höger snedfördelad dock inte sä mycket samt det finns en del av outliers åt höger. Om vi ska log-transformera target bli en distribution normalfördelad.

Vi ska kolla på några features som är intresanta. Variablerna 10, 20 och 23 ser ut som årtal. Vi kan kolla på hur deras distribution ser ut.

In [8]:
fig = px.histogram(df, x=["feature10", "feature23",'feature20'], y="target", opacity=0.7, nbins=20,
                   color_discrete_sequence=['steelblue', 'salmon','cadetblue'],
                   title = 'Distribution of Feature10, Feature20 and Feature23') 
fig.update_layout(legend=dict(
    orientation='h',
    yanchor="bottom",
    y=-0.3,
    xanchor="center",
    x=0.5
    )
)
fig.update_xaxes(title_text=None)
fig

In [9]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Box(x=df['feature15'], y=df['target'], name='feature15'
                    ),
              1, 1)

fig.add_trace(go.Box(x=df['feature16'], y=df['target'], name='feature16'
                    ),
              1, 2)

fig.update_layout(legend=dict(
    orientation='h',
    yanchor="bottom",
    y=-0.3,
    xanchor="center",
    x=0.5
    )
)
fig.update_yaxes(title_text='Target')
fig.update_layout(title_text='Boxplot of Feature15 and Feature16')
fig

Feature 15 och feature 16 ser ut likadana. T.ex. kategorien 'amazing' har samma median och samma max värde. Kategorien 'good' är också ganska lika. Det kan betyda att de två variblerna kan har en hög korrelationskoefficient. Om det är så blir det smidigt att exkludera en av de från vår model. 

In [10]:
df.groupby(['feature15','feature16'])[['target']].count().sort_values('target', ascending=False)

target
feature15 feature16        
good      good          358
bad       bad           290
          good          287
good      bad           231
          amazing       144
okay      bad            48
horrible  bad            36
okay      good           27
amazing   amazing        24
          good           18
bad       amazing        17
good      okay           11
horrible  good            9
bad       okay            6
amazing   bad             1
horrible  okay            1

### Missing values

In [11]:
nan = pd.DataFrame({
    'Missing Values': df.isnull().sum().sort_values(ascending=False),
    'Missing Values, %': 
        (df.isnull().sum().sort_values(ascending=False)/len(df) * 100).astype(int)
})
nan[nan['Missing Values'] > 0]

,Missing Values,"Missing Values, %"
feature11,2732,93
feature12,2358,80
feature15,1422,48
feature02,1,0
feature04,1,0
feature01,1,0
feature13,1,0


Variblerna 'feature11' och 'feauture12' har 93% och 80% av saknade värdena. 'Feature15' har hälften av saknade observationerna. Vi har redan undersökt att 'feature15' och 'feature16' är överlappande därför kan vi radera 'feature15'.

Vi ska göra en kopia av våra data innan vi raderar kolumner samt vi ska kolla på saknade värde i 'feature01', 'feature02', 'feature04' och 'feature13' för att välja en rätt strategi för att ersätta de saknade värdena.

In [12]:
df_pred = df.copy()
df_pred.drop(['feature11','feature12','feature15'], axis=1, inplace=True)
df_pred[df_pred.isna().any(axis=1)]

,target,feature01,feature02,feature03,feature04,feature05,feature06,feature07,feature08,feature09,feature10,feature10.1,feature13,feature14,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23
id,,,,,,,,,,,,,,,,,,,,,,
1342,79000,1.0,280.0,0,NaN,896,4,896,4,7,1946,1,blue,0,bad,2,1,4,2008,0,0,1950
1578,167500,2.0,400.0,0,384.0,754,7,1394,5,5,2006,2,NaN,0,good,3,1,5,2008,1,640,2007
2237,150909,NaN,NaN,0,859.0,942,6,1828,5,6,1923,2,red,0,good,3,1,3,2007,0,886,1999


https://towardsdatascience.com/imputing-missing-values-using-the-simpleimputer-class-in-sklearn-99706afaff46   

https://campus.datacamp.com/courses/dealing-with-missing-data-in-python/imputation-techniques?ex=2

In [13]:
freq_imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
df_pred[['feature13']] = freq_imputer.fit_transform(df_pred[['feature13']])
med_imputer = SimpleImputer(strategy = 'median', missing_values=np.nan)
df_pred[['feature01','feature02','feature04']] = med_imputer.fit_transform(df_pred[['feature01','feature02','feature04']])

In [14]:
df_pred.loc[[1342, 2237, 1578], ['feature01', 'feature02', 'feature04', 'feature13']]

,feature01,feature02,feature04,feature13
id,,,,
1342,1.0,280.0,990.0,blue
2237,2.0,480.0,859.0,red
1578,2.0,400.0,384.0,red


Det finns kvar två kategoriska variabler, 'feature13' och 'feature16'. Vi ska transformera dem til s.k. dummyvariabler

In [15]:
df_pred = pd.get_dummies(df_pred, drop_first=True)

## Train-test-split

In [16]:
X = df_pred.drop('target', axis = 1)
y = df_pred['target']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 17)

## The Null Model

In [17]:
y_pred_null = y_train.mean()
pred_train_null = np.repeat(y_pred_null, y_train.size)
pred_test_null = np.repeat(y_pred_null, y_test.size)
R2_train_null = R2(y_train, pred_train_null).round(4)
R2_test_null = R2(y_test, pred_test_null).round(4)
RMSE_train_null = int(MSE(y_train, pred_train_null,squared=False))
RMSE_test_null = int(MSE(y_test, pred_test_null,squared=False))
MAE_train_null = int(MAE(y_train, pred_train_null))
MAE_test_null = int(MAE(y_test, pred_test_null))

Skapar en dataframe där alla metrics ska sparas och lägger null-modell metrics till den.

In [18]:
df_metrics = pd.DataFrame({
    "metric" : ['R2_train', 'R2_test', 
                    'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    'null' : [R2_train_null, R2_test_null, RMSE_train_null, 
                RMSE_test_null, MAE_train_null, MAE_test_null]}).set_index('metric', drop=True)

## Linear Regression Model with all features and Log-transformed target

In [19]:
lr = LinearRegression()
y_train_log, y_test_log = np.log(y_train), np.log(y_test)

lr.fit(X_train, y_train_log)

metrics_df('LR_log_all_f', lr, log='y')
output_metrics('LR_log_all_f')

Training R2: 0.8528 vs Test R2: 0.8738
Training RMSE: 44050 vs Test RMSE: 25337
Training MAE: 18841 vs Test MAE: 17436


Linear Regression modell visar ganska bra resultat. Dock vi ska kolla om vi kan förbättra vår model. Det kan vi göra med feature engineering and feature selection. 
Först ska vi kolla på vilka variabler är mer eller mindre relevanta för vår model. Samtidigt ska vi jämföra linjära koefficienter med korrelationskoefficienter och 'feature impotances' från Random Forest Regressor.

In [20]:
cormat = df_pred.corr().iloc[1:,0].reset_index()
rfr = RandomForestRegressor(n_estimators=150, max_depth=16, max_features='sqrt', random_state=1111)
rfr.fit(X_train,y_train)

RandomForestRegressor(max_depth=16, max_features='sqrt', n_estimators=150,
                      random_state=1111)

In [21]:
importances = pd.DataFrame({
    'Feature': X.columns,
    'Linear coefficient': lr.coef_,
    'RF feature importance': rfr.feature_importances_,
    'Correlation coef' : cormat.iloc[:,1]

})
importances = importances.sort_values(by='RF feature importance', ascending=False)

https://plotly.com/python/subplots/

https://stackoverflow.com/questions/62982784/plotly-bar-chart-change-color-based-on-positive-negative-value-python

In [22]:
importances['Color_reg'] = np.where(importances['Linear coefficient'] < 0, 'gold', 'steelblue')
importances['Color_cor'] = np.where(importances['Correlation coef'] < 0, 'salmon', 'cadetblue')

fig = make_subplots(rows=1, cols=3, shared_yaxes=True,  
                    subplot_titles=('Random Forest', 'Linear Regression', 'Correlation'))

fig.add_trace(go.Bar(y=importances['Feature'], x=importances['Correlation coef'],orientation='h',
                  marker_color=importances['Color_cor']
                    ),
              1, 3)

fig.add_trace(go.Bar(y=importances['Feature'], x=importances['RF feature importance'],orientation='h',
                  marker_color='lightslategray'
                    ),
              1, 1)

fig.add_trace(go.Bar(y=importances['Feature'], x=importances['Linear coefficient'],orientation='h', 
            marker_color=importances['Color_reg']
                   ),
              1, 2)

fig.update_layout(height=600, 
                  title_text = "Weight of Each Feature for Predicting the Target and the Correlation Coefficients", 
                  template = 'seaborn', showlegend=False)
fig


Som vi ser att resultat är olika beroende på modellen. Om man ska välja Random Forest Regression är 'feauture16_okay' inte viktigt alls dock den är viktigt om man skapar Linear Regression. Dock har variable 'feature08' en högt vikt både för Linear Regression och Random Forest samt har en högt samband med target, 0.8. Samma gäller 'feauture01' som har en korrelationskoefficient 0.65, och vikten 0.1 och 0.05 för LR respektive RF.

Medan 'tree-based models' är inte känsliga mot outliers och multikollinearitet, är Linjär Regression känslig mot dem. Vi ska kolla på om två (eller flera) kovarianser är högt korrelerade. Det finns olika tumregler när VIF-värdet är för högt: när VIF > 10 och när VIF > 5 (Borg E, Westerlund J. Statistik för beteendevetare, 2012, s.410)

https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python

In [23]:
df_mult = X.assign(const=1)
vif = pd.DataFrame(
    {"Feature" : df_mult.columns,
    "VIF Factor" : [int(VIF(df_mult.values, i)) for i in range(df_mult.shape[1])]
     })
vif[vif['VIF Factor'] > 5].sort_values(by='VIF Factor', ascending=False).iloc[1:,]

,Feature,VIF Factor
6,feature07,127
17,feature22,91
4,feature05,78
23,feature16_bad,7


Vi kan kolla på korellation mellan target, 'feature04', 'feature05', 'feature07'.

In [24]:
df_pred[["target","feature04", "feature05",'feature07']].corr()

,target,feature04,feature05,feature07
target,1.000000,0.632164,0.621676,0.706780
feature04,0.632164,1.000000,0.800688,0.444622
feature05,0.621676,0.800688,1.000000,0.562166
feature07,0.706780,0.444622,0.562166,1.000000


Som vi ser att 'feature04' och 'feature05' har en stark samband, 0.8. Medan 'feature07' har lite högre samband med en target, 0.7.
Vi ska excludera 'feature05'. Samtidigt ska vi transformera variblerna 'feature10', 'feature20' och 'feature23'. Vi skapar två nya variabler som är subtraktion 'feature10' och 'feature23' från 'feature20'.
Variablerna 'feature18', 'feature21' och 'feature22' ska också exkluderas. 

In [25]:
df_feature_select = df_pred.copy()
df_feature_select['diff_f20_f10'] = df_feature_select['feature20'] - df_feature_select['feature10']
df_feature_select['diff_f20_f23'] = df_feature_select['feature20'] - df_feature_select['feature23']
col_to_drop = ['feature05', 'feature10','feature18','feature20','feature21','feature22','feature23']
df_feature_select.drop(col_to_drop, axis = 1, inplace=True)

In [26]:
X = df_feature_select.drop('target', axis = 1)
y = df_feature_select['target']
X_scaled = RobustScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size = 0.3, random_state = 17)

In [27]:
df_mult = X.assign(const=1)
vif = pd.DataFrame(
    {"Feature" : df_mult.columns,
    "VIF Factor" : [int(VIF(df_mult.values, i)) for i in range(df_mult.shape[1])]
     })
vif[vif['VIF Factor'] > 5].sort_values(by='VIF Factor', ascending=False).iloc[1:,]

,Feature,VIF Factor
16,feature16_bad,7


## Gradient Boosting for regression

In [46]:
parameters = {
            'n_estimators' : [100, 150, 300, 500],
            "max_depth": [1, 3, 5, 10],
            "max_features": ['auto', 'sqrt']
            }

gbr = GradientBoostingRegressor(random_state=1111)
gbr.fit(X_train, y_train)

gbr_search =GridSearchCV(
        estimator=gbr,
        param_grid=parameters,
        cv=5,
        scoring = 'r2',
        n_jobs=-1)
        
gbr_search.fit(X_train, y_train)
gbr_search.best_params_

{'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 300}

In [47]:
metrics_df('GrBoostingReg', gbr_search.best_estimator_)
output_metrics('GrBoostingReg')

Training R2: 0.9560 vs Test R2: 0.9125
Training RMSE: 16778 vs Test RMSE: 23517
Training MAE: 12168 vs Test MAE: 16705


## Random Forest Regression

In [31]:
param_dist = {
            'n_estimators' : [100, 150, 175, 300],
            "max_depth": [12, 14, 16, 18],
            "max_features": ['auto', 'sqrt']
            }

rfr = RandomForestRegressor(random_state=1111)
rfr.fit(X_train, y_train)

rbr_search = GridSearchCV(
        estimator=rfr,
        param_grid=param_dist,
        cv=5,
        scoring='r2',
        n_jobs=-1)

rbr_search.fit(X_train, y_train)
rbr_search.best_params_

{'max_depth': 16, 'max_features': 'sqrt', 'n_estimators': 150}

In [32]:
metrics_df('RandForestReg', rbr_search.best_estimator_)
output_metrics('RandForestReg')

Training R2: 0.9820 vs Test R2: 0.8942
Training RMSE: 10732 vs Test RMSE: 25860
Training MAE: 7020 vs Test MAE: 17811


## The final model. Stacking regression

Sista modellen är Stacking Regression. Vi ska kombinera predikationer från Gradient Boosting och Random Forest och använda RidgeCV som 'a meta-learning algorithm'. https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/

In [48]:
estimators = [('rf', rbr_search.best_estimator_),
                ('gb', gbr_search.best_estimator_)]

sr = StackingRegressor(estimators=estimators, final_estimator = RidgeCV(), cv=5)
sr.fit(X_train, y_train)
metrics_df('StackingRegr', sr)
output_metrics('StackingRegr')

Training R2: 0.9711 vs Test R2: 0.9134
Training RMSE: 13605 vs Test RMSE: 23404
Training MAE: 9920 vs Test MAE: 16560


### All model's metrics

In [34]:
df_metrics

,null,LR_log_all_f,GrBoostingReg,RandForestReg,StackingRegr
metric,,,,,
R2_train,0.0,0.8528,0.9560,0.9820,0.9711
R2_test,-0.0,0.8738,0.9125,0.8942,0.9134
RMSE_train,80025.0,44050.0000,16778.0000,10732.0000,13605.0000
RMSE_test,79516.0,25337.0000,23517.0000,25860.0000,23404.0000
MAE_train,57971.0,18841.0000,12168.0000,7020.0000,9920.0000
MAE_test,59029.0,17436.0000,16705.0000,17811.0000,16560.0000


Vi ser att Gradient Boosting Regression visar mycket bättre resultat än Linjär Regression. Random Forest Regression har sämre resultat är Gradient Boosting och överfit för mycket. Den bästa resultatet visar Stacking Regression.

## Scatter plot predicted values vs actual values

https://plotly.com/python/ml-regression/

In [35]:
fig = px.scatter(x=y_test, y=sr.predict(X_test), opacity=0.7,
                title = 'Predicted Values vs Actual Values with linear fit')
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=y.min(), y0=y.min(),
    x1=y.max(), y1=y.max()
)
fig.update_xaxes(title_text = 'Actual Value')
fig.update_yaxes(title_text = 'Predicted Value')
fig

Vi skapar en dataframe med prediktioner och faktiska värdena samt residuals (error) för test dataset.

In [36]:
final_y_pred = sr.predict(X_test)
residuals = y_test - final_y_pred
result_df = pd.DataFrame({
    "y_actual_test" : y_test,
    'y_predict_test' : final_y_pred,
    'residuals_test' : residuals})
result_df.astype(int).head(3)

,y_actual_test,y_predict_test,residuals_test
id,,,
2396,360000,391961,-31961
1816,128000,129536,-1536
192,128500,139577,-11077


https://stackoverflow.com/questions/61440583/python-plotly-different-axis-title-in-subplot-marginal

In [37]:
fig = px.histogram(x=result_df['y_actual_test'], y=result_df['residuals_test'],
                   color_discrete_sequence=['steelblue'], marginal="histogram", opacity=0.7,
                   title = 'Distribution of Residuals and Actual Target')
fig.update_xaxes(title_text='Actual Target',row=1, col=1)
fig.update_yaxes(title_text='Sum of Residuals', row=1, col=1)
fig.update_traces(marker=dict(color="lightsteelblue"),row=2, col=1)
fig

Här ser vi ganska intressant mönster. Om target är mellan 0 och 200 000 är prediktionerna högre än faktiska värdena (obs. det är summan of residuals, vissa predikationer kan bli lägre än target i denna interval) och tvärtom.

Vi ska radera de residuals som är lägre än RMSE_test för Stacking Regression modell för att undvika overplotting.

In [39]:
result_trimmed = result_df[(result_df['residuals_test'] > df_metrics.iloc[3,4]) | (result_df['residuals_test'] < -df_metrics.iloc[3,4])]

https://medium.com/@caiotaniguchi/plotting-lollipop-charts-with-plotly-8925d10a3795

In [84]:
data = [
    go.Scatter(
        x=result_trimmed['y_predict_test'],
        y=result_trimmed['residuals_test'],
        mode='markers',
        marker=dict(color='red')
    )
]

fig = go.Layout(
    shapes=[dict(
        type='line',
        xref='x',
        yref='y',
        x0 = result_trimmed['y_predict_test'].iloc[i],
        y0 = result_trimmed['residuals_test'].iloc[i],
        x1 = result_trimmed['y_predict_test'].iloc[i],
        y1 = 0,
        line = dict(
            color = 'grey',
            width = 0.5
        )
    ) for i in range(len(result_trimmed['residuals_test']))],
    title='Residuals for Stacking Regression Model'
)

fig = go.Figure(data, fig)
fig.update_xaxes(title_text = 'Predicted Value')
fig.update_yaxes(title_text = 'Residuals')
fig

Vi kan kolla på vissa obserbationer för att jämföra dem. T.ex, observation 957 och 2884 har samma target, 375 000, dock vår modell predikterar olika värden, 223 599 resp. 279 497. Detta är intressant. Det kan bli variablerna 'diff_f20_f10' och 'diff_f20_f23' som orsakar resultat. 
Observationer 1826 och 387 har lägsta residuals, 12 resp. -39. Deras faktiska värden är 127 500 och 235 000. Och igen, vi ser att 'diff_f20_f10' och 'diff_f20_f23' skijer sig ganska mycket! I detta fall, kan vi säja att kanske var det inte 'diff_f20_f10' och 'diff_f20_f23' i första fallet som orsakade predikationer. 
Det vore intressant att undersöka vidare variablerna för att skapa en bättre modell. 

In [41]:
result_df.loc[[387, 957, 1826, 2884], :].astype(int).merge(X.loc[[387, 957, 1826, 2884], :], 
                on='id').sort_values('y_actual_test')

,y_actual_test,y_predict_test,residuals_test,feature01,feature02,feature03,feature04,feature06,feature07,feature08,feature09,feature10.1,feature14,feature17,feature19,feature13_green,feature13_pink,feature13_red,feature13_yellow,feature16_bad,feature16_good,feature16_horrible,feature16_okay,diff_f20_f10,diff_f20_f23
id,,,,,,,,,,,,,,,,,,,,,,,,,
1826,128500,128487,12,2.0,572.0,0,988.0,5,988,5,5,1,0,3,10,0,0,1,0,1,0,0,0,46,46
387,235000,235039,-39,2.0,565.0,0,972.0,7,1811,8,5,2,1,3,10,0,0,1,0,0,1,0,0,7,7
957,375000,223599,151400,2.0,513.0,0,2136.0,8,2036,7,5,2,2,3,6,0,0,1,0,1,0,0,0,44,44
2884,375000,279497,95502,2.0,495.0,0,1573.0,5,1778,8,5,2,1,2,2,0,0,1,0,0,0,0,0,1,0


# Conclusion

Den sluttliga modell är Stacking Regression, meta-algorithm som kombinerar predikationer från Gradient Boosting Regression och Random Forest Regression och använder 'Ridge regression with built-in cross-validation' som 'final estimator' för göra predikationer.
Modellen visar ganska högt R squared både på train datan och test datan, 0.97 resp. 0.91. Dock finns det överfitting, modellen predikterar bättre på train data än på test data. 
Jag har användat Robust Scaler för alla modeller. Det ge lite förbättring på predikationerna dock inte så mycket. 

Jag har testat andra modeller, tex K-nearest Neighbors Regression. Dessutom har jag testat samma Stacking Regression med alla variabler.

parametrar för KNN Regressor {'algorithm': 'ball_tree', 'metric': 'minkowski', 'n_neighbors': 12, 'weights': 'uniform'}

parametrar för Stacking Regression:

Random Forest Regressor {'max_depth': 18, 'max_features': 'sqrt', 'n_estimators': 300}

Gradient Boosting Regressor {'max_depth': 3, 'max_features': 'auto', 'n_estimators': 150}

RobustScaler()

In [70]:
df_other_models = pd.read_csv('other_models')
df_compare = df_metrics[['StackingRegr','LR_log_all_f']].merge(df_other_models, on='metric')
df_compare = df_compare.reindex(['knn_sel_f', 'LR_log_all_f', 'LR_log_sel_f','StackingRegr','StackingRegr_all_f'], axis=1)
df_compare

,knn_sel_f,LR_log_all_f,LR_log_sel_f,StackingRegr,StackingRegr_all_f
0,0.8570,0.8528,0.8504,0.9711,0.9679
1,0.8633,0.8738,0.8722,0.9134,0.9158
2,30258.0000,44050.0000,44091.0000,13605.0000,14327.0000
3,29404.0000,25337.0000,25472.0000,23404.0000,23069.0000
4,18425.0000,18841.0000,19126.0000,9920.0000,10320.0000
5,19857.0000,17436.0000,17533.0000,16560.0000,16194.0000


Och resultatet är bättre! 

Det är lite svårt att bedöma om det bästa resultat är använbar eftersom jag inte vet om 'feature10', 'feature20' och 'feature23' är faktiska årtal eller inte. 

In [81]:
result_df_all_mod = pd.read_csv('residuals_all_f')

Vi kan jämföra prediktioner från både modeller och plotta dem.

In [176]:
all_result = result_df_all_mod.merge(result_df, on='id', suffixes=['_all','_sel']).astype(int)

In [178]:
fig = px.scatter(all_result, x='y_actual_test_all', y=['y_predict_test_all','y_predict_test_sel'], opacity=0.7,
                title = 'Predicted Values vs Actual Values with linear fit')

fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=y.min(), y0=y.min(),
    x1=y.max(), y1=y.max()
)
fig.update_layout(legend=dict(
    orientation='h',
    yanchor="bottom",
    y=-0.3,
    xanchor="center",
    x=0.5
    )
)
fig.update_xaxes(title_text = 'Actual Value')
fig.update_yaxes(title_text = 'Predicted Value')
fig